# Excel Tabs to PowerPoint Slides (One Tab -> One Slide)

This notebook reads each worksheet tab and adds it to a separate PowerPoint slide as a scaled image so varying tab sizes/types are handled consistently.

In [ ]:
# Step 1: (Optional) Install packages in a fresh environment.
# Uncomment and run once if needed.
# !pip install pandas openpyxl python-pptx matplotlib

In [ ]:
# Step 2: Import libraries.
from pathlib import Path
import tempfile

import pandas as pd
import matplotlib.pyplot as plt
from pptx import Presentation
from pptx.util import Inches, Pt

In [ ]:
# Step 3: Define paths.
# This works whether the notebook runs from /Sandbox or /Sandbox/Scripts.
cwd = Path.cwd()
project_root = cwd.parent if cwd.name == 'Scripts' else cwd
scripts_dir = project_root / 'Scripts'

# Preferred source file (Excel).
excel_path = scripts_dir / 'sample excel termplate.xlsx'

output_pptx_path = scripts_dir / 'excel_tabs_to_powerpoint_output.pptx'

print(f'Using Excel file: {excel_path.resolve()}')
print(f'Will save PowerPoint to: {output_pptx_path.resolve()}')

In [ ]:
# Step 4: Read worksheet tabs and optionally filter to a defined tab list.
# sheet_name=None returns a dict: {tab_name: DataFrame}.
all_sheets_raw = pd.read_excel(excel_path, sheet_name=None, dtype=object)

# Optional allowlist of tabs (exact names). Keep [] to include all tabs.
selected_tabs = [
    # 'Summary',
    # 'Transactions',
]

if selected_tabs:
    missing_tabs = [t for t in selected_tabs if t not in all_sheets_raw]
    if missing_tabs:
        raise ValueError(f'These tabs were not found in the workbook: {missing_tabs}')

    # Preserve the allowlist order in final slide order.
    all_sheets = {tab: all_sheets_raw[tab] for tab in selected_tabs}
else:
    all_sheets = all_sheets_raw

for sheet_name, df in all_sheets.items():
    print(f'{sheet_name}: {df.shape[0]} rows x {df.shape[1]} columns')

In [ ]:
# Step 5: Build helper to render one sheet as an image.
# Rendering to image allows one-slide-per-tab, even when dimensions vary.
def render_sheet_to_image(df, image_path):
    # Keep all values and convert to strings for consistent rendering.
    display_df = df.copy().fillna('').astype(str)

    # If a tab is fully empty, create a tiny placeholder frame.
    if display_df.empty and len(display_df.columns) == 0:
        fig, ax = plt.subplots(figsize=(8, 2))
        ax.axis('off')
        ax.text(0.5, 0.5, 'Empty sheet', ha='center', va='center', fontsize=14)
        fig.savefig(image_path, dpi=220, bbox_inches='tight')
        plt.close(fig)
        return

    n_rows = max(1, len(display_df))
    n_cols = max(1, len(display_df.columns))

    # Dynamic canvas sizing so wider/taller tabs get more space.
    fig_width = min(26, max(10, n_cols * 1.5))
    fig_height = min(18, max(3, (n_rows + 1) * 0.35))

    fig, ax = plt.subplots(figsize=(fig_width, fig_height))
    ax.axis('off')

    table = ax.table(
        cellText=display_df.values,
        colLabels=[str(c) for c in display_df.columns],
        cellLoc='left',
        loc='center'
    )

    # Adaptive font size for different tab sizes.
    font_size = max(6, min(12, int(220 / n_rows), int(160 / n_cols)))
    table.auto_set_font_size(False)
    table.set_fontsize(font_size)
    table.scale(1, 1.15)

    fig.savefig(image_path, dpi=220, bbox_inches='tight')
    plt.close(fig)

In [ ]:
# Step 6: Build helper to add one image slide per tab.
def add_image_slide(prs, title_text, image_path):
    slide = prs.slides.add_slide(prs.slide_layouts[6])  # Blank

    # Slide title
    title_box = slide.shapes.add_textbox(Inches(0.5), Inches(0.2), Inches(12.3), Inches(0.6))
    title_frame = title_box.text_frame
    title_frame.text = title_text
    title_frame.paragraphs[0].runs[0].font.size = Pt(24)

    # Slide content area (below title)
    left = Inches(0.4)
    top = Inches(1.0)
    max_width = Inches(12.5)
    max_height = Inches(5.9)

    # Add image and constrain to content area.
    pic = slide.shapes.add_picture(str(image_path), left, top, width=max_width)

    if pic.height > max_height:
        # Re-add with height constraint if width-based placement is too tall.
        pic._element.getparent().remove(pic._element)
        slide.shapes.add_picture(str(image_path), left, top, height=max_height)

In [ ]:
# Step 7: Create PowerPoint with one slide per worksheet tab.
prs = Presentation()

with tempfile.TemporaryDirectory() as temp_dir:
    temp_dir_path = Path(temp_dir)

    for idx, (sheet_name, df) in enumerate(all_sheets.items(), start=1):
        image_path = temp_dir_path / f'sheet_{idx}.png'

        # Render the full sheet data to an image, then place on its own slide.
        render_sheet_to_image(df, image_path)
        add_image_slide(prs, sheet_name, image_path)

# Save final deck.
prs.save(output_pptx_path)
print(f'PowerPoint created: {output_pptx_path.resolve()}')